In [14]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows
EXISTING_CHECKPOINT_KAGGLE_DATASET_ID = "hsm-models" # set to None if you want to train a new model on Kaggle. Else, set to the Kaggle dataset ID where the existing model checkpoints are stored
PRETRAINED_MODEL_NAME_OR_PATH = "tf_efficientnet_b0_ns"

In [15]:
import os
import sys

from tqdm import tqdm
import torch

import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader


if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.utils.utils import get_raw_data_dir, get_submission_csv_path, get_models_save_path, set_seeds
from src.utils.constants import Constants

from src.datasets.spectrogram_dataset import SpectrogramDataset
from src.models.base_cnn import BaseCNN

set_seeds(Constants.SEED)

In [16]:
DATA_PATH = get_raw_data_dir()
test_df = pd.read_csv(DATA_PATH / "test.csv")

In [18]:
test_df.head()

,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [19]:
class InfCFG:
    data_path = DATA_PATH
    model_name = PRETRAINED_MODEL_NAME_OR_PATH
    target_size = 6
    img_size = (128, 256)
    n_splits = 5
    batch_size = 64
    num_workers = 8
    model_dir = get_models_save_path(EXISTING_CHECKPOINT_KAGGLE_DATASET_ID) / "base_cnn" / PRETRAINED_MODEL_NAME_OR_PATH / DATA_PREPARATION_VOTE_METHOD

InfCFG.model_paths = [os.path.join(InfCFG.model_dir, f'best_model_fold{i}.pth') for i in range(InfCFG.n_splits)]

In [20]:
def run_inference():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    test_df = pd.read_csv(os.path.join(InfCFG.data_path, 'test.csv'))
    test_dataset = SpectrogramDataset(
        df=test_df, 
        targets=Constants.TARGETS, 
        data_path=InfCFG.data_path, 
        img_size=InfCFG.img_size, 
        mode='test'
    )
    test_loader = DataLoader(
        test_dataset, batch_size=InfCFG.batch_size, shuffle=False, num_workers=InfCFG.num_workers
    )

    all_fold_predictions = []

    for i, path in enumerate(InfCFG.model_paths):
        print(f"\n========== Inferencing with Fold {i} Model ==========")
        if not os.path.exists(path):
            print(f"Model file not found: {path}. Skipping this fold.")
            continue
            
        model = BaseCNN(InfCFG.model_name, pretrained=False, num_classes=InfCFG.target_size)
        model.load_state_dict(torch.load(path, map_location=device))
        model.to(device)
        model.eval()

        current_fold_preds = []
        with torch.no_grad():
            for images in tqdm(test_loader, desc=f"Predicting Fold {i}"):
                outputs = model(images.to(device))
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                current_fold_preds.append(probs)
        
        all_fold_predictions.append(np.concatenate(current_fold_preds))

    if not all_fold_predictions:
        print("No models were found for inference. Aborting.")
        return

    avg_predictions = np.mean(all_fold_predictions, axis=0)
    submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
    submission[Constants.TARGETS] = avg_predictions

    submission.to_csv(get_submission_csv_path(), index=False)

In [21]:
run_inference()

Using device: cuda

========== Inferencing with Fold 0 Model ==========


/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(
Predicting Fold 0: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]



========== Inferencing with Fold 1 Model ==========


Predicting Fold 1: 100%|██████████| 1/1 [00:00<00:00,  3.03it/s]



========== Inferencing with Fold 2 Model ==========


Predicting Fold 2: 100%|██████████| 1/1 [00:00<00:00,  3.01it/s]



========== Inferencing with Fold 3 Model ==========


Predicting Fold 3: 100%|██████████| 1/1 [00:00<00:00,  3.06it/s]



========== Inferencing with Fold 4 Model ==========


Predicting Fold 4: 100%|██████████| 1/1 [00:00<00:00,  3.06it/s]
